In [1]:
# This Notebook Enriches the Joins the Officers and Teams tablue using the user teams mapping

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
2,None,pyspark,idle,,,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
import sys
sys.argv.append('--source_catalog_table')
# TODO: set name of the source table name in the glue catalog 
sys.argv.append('dtf_locations')
# changed to full file

sys.argv.append('--source_catalog_table2')
# TODO: set name of the source table name in the glue catalog 
sys.argv.append('unrestricted_address_api_dbo_hackney_address')
# Changed to full file

sys.argv.append('--source_catalog_database')
# TODO: set name of the source database name in the glue catalog 
sys.argv.append('dataplatform-stg-tascomi-refined-zone')

sys.argv.append('--source_catalog_database2')
# TODO: set name of the source database name in the glue catalog 
sys.argv.append('dataplatform-stg-raw-zone-unrestricted-address-api')

sys.argv.append('--s3_bucket_target')
# TODO: set S3 location where the transformed data will be saved
sys.argv.append('s3://dataplatform-stg-trusted-zone/tascomi/tables')


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
import sys
from awsglue.transforms import *
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.dynamicframe import DynamicFrame
from awsglue.job import Job
from awsglue.utils import getResolvedOptions
from pyspark.sql.functions import *
import pyspark.sql.functions as F

# from helper.helpers import get_glue_env_var,

def get_glue_env_var(key, default="none"):
    if f'--{key}' in sys.argv:
        return getResolvedOptions(sys.argv, [key])[key]
    else:
        return default

glueContext = GlueContext(SparkContext.getOrCreate())
job = Job(glueContext)


# Define Source Data
source_catalog_table = get_glue_env_var('source_catalog_table', '')
source_catalog_table2 = get_glue_env_var('source_catalog_table2', '')

source_catalog_database = get_glue_env_var('source_catalog_database', '')
source_catalog_database2 = get_glue_env_var('source_catalog_database2', '')

target_bucket = get_glue_env_var('s3_bucket_target', '')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
def drop_null_columns(df):
  
    _df_length = df.count()
    null_counts = df.select([F.count(F.when(F.col(c).isNull(), c)).alias(c) for c in df.columns]).collect()[0].asDict()
    to_drop = [k for k, v in null_counts.items() if v >= _df_length]
    df = df.drop(*to_drop)
    
    return df

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
def get_latest_snapshot(df):
    
   df = df.where(col('snapshot_date') == df.select(max('snapshot_date')).first()[0])
   return df

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
def get_latest_import(df):
    
   df = df.where(col('import_date') == df.select(max('import_date')).first()[0])
   return df

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
# Load DTF Locations Table
data_source = glueContext.create_dynamic_frame.from_catalog(
    name_space=source_catalog_database,
    table_name=source_catalog_table
 )

# convert to a data frame
df = data_source.toDF()


# Rename columns
df = df.withColumnRenamed("id","dtf_location_id") \


df = get_latest_snapshot(df)

# Drop Columns containing Only Nulls 
# df = drop_null_columns(df)

# Create Calculated Fields for Reporting

df = df.withColumn('counter_location', lit(1))



FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
mapWard = {
            "Hoxton East and Shoreditch": "Hoxton East & Shoreditch"
            }
    

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
# Load Address base Hackney Addresses Table
data_source2 = glueContext.create_dynamic_frame.from_catalog(
    name_space=source_catalog_database2,
    table_name=source_catalog_table2
)

# convert to a data frame
df2 = data_source2.toDF()

# Get latest import

df2 = get_latest_import(df2)

# Keep Only Relevant Columns
df2 = df2.select("uprn",
                 "blpu_class",
                 "lpi_key",
                 "lpi_logical_status",
                 "ward",
                 "postcode",
                 "line1",
                 "line2",
                 "line3",
                 "line4")

#create combined address field
df2 = df2.withColumn('llpg_address',concat(trim(col('line1')),
                                         lit(", "),
                                         trim(col('line2')),
                                         lit(", "),
                                         trim(col('line3')),
                                         lit(", "),
                                         trim(col('line4')),
                                         lit(", "),
                                         trim(col('postcode')),
                                        ))


# Rename Relevant Columns
df2 = df2.withColumnRenamed("lpi_key","llpg_lpi_key") \
         .withColumnRenamed("blpu_class","llpg_blpu_class") \
         .withColumnRenamed("uprn","llpg_uprn") \
         .withColumnRenamed("lpi_logical_status","llpg_logical_status") \
         .withColumnRenamed("ward","llpg_ward") \
         .withColumnRenamed("postcode","llpg_postcode") \
         .withColumnRenamed("line1","llpg_line1") \
         .withColumnRenamed("line2","llpg_line2") \
         .withColumnRenamed("line3","llpg_line3") \
         .withColumnRenamed("line4","llpg_line4") \

# Duplicate ward column to apply mapping

df2 = df2.withColumn('llpg_ward_map', col('llpg_ward').cast('string')) \

# Apply Mapping
df2 = df2.replace(to_replace=mapWard, subset=['llpg_ward_map'])



# Check Result
# df2.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
# Join

df = df.join(df2,df.uprn ==  df2.llpg_uprn,"left")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
# Convert data frame to dynamic frame 
dyanmic_frame = DynamicFrame.fromDF(df, glueContext, "target_data_to_write")

# Write the data to S3
parquet_data = glueContext.write_dynamic_frame.from_options(
    frame=dynamic_frame,
    connection_type="s3",
    format="parquet",
    connection_options={"path": target_bucket, "partitionKeys": ["import_year", "import_month", "import_day", "import_date"]},
    transformation_ctx="target_data_to_write")